# TerraClimate V2 - Extraction avec Agrégations Temporelles

## Améliorations par rapport à V1

**V1** : Extrait uniquement la valeur du mois de mesure

**V2** : Ajoute des **agrégations temporelles** pour capturer l'historique climatique :

| Feature | Description | Pourquoi c'est utile |
|---------|-------------|---------------------|
| `ppt` | Précipitations du mois | Valeur actuelle |
| `ppt_lag1` | Précipitations mois-1 | Ruissellement différé |
| `ppt_lag2` | Précipitations mois-2 | Impact retardé |
| `ppt_sum3` | Cumul 3 derniers mois | Saturation du sol |
| `ppt_anomaly` | Écart à la moyenne saisonnière | Mois exceptionnellement pluvieux/sec |

## Note importante

TerraClimate est **mensuel** (pas journalier). Pour des cumuls 3j/7j, il faudrait utiliser ERA5 ou CHIRPS.
Ici on fait des **lags mensuels** qui capturent quand même l'historique climatique récent.

---

## Étape 1 : Installation et imports

In [1]:
!pip install uv
!uv pip install --system -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
Using Python 3.13.1 environment at: c:\Program Files\Python313
Resolved 194 packages in 3.82s
 Downloaded pywinpty
Prepared 2 packages in 359ms
error: Failed to install: pywinpty-3.0.3-cp313-cp313-win_amd64.whl (pywinpty==3.0.3)
  Caused by: failed to create directory `c:\Program Files\Python313\Lib\site-packages\pywinpty-3.0.3.dist-info`: Accès refusé. (os error 5)


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import xarray as xr
from scipy.spatial import cKDTree
import pystac_client
import planetary_computer as pc
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import os

print("Imports OK!")

Imports OK!


---

## Étape 2 : Configuration

In [3]:
# =============================================================================
# CONFIGURATION
# =============================================================================

OUTPUT_DIR = "../data/processed"

# Variables de base à extraire
TERRACLIMATE_VARIABLES = ['pet', 'aet', 'ppt', 'tmax', 'tmin', 'soil', 'def', 'pdsi', 'vpd', 'ws']

# Variables pour lesquelles on veut des lags et cumuls
TEMPORAL_VARS = ['ppt', 'soil', 'def', 'vpd']

# Nombre de mois de lag
N_LAGS = 3

print(f"Variables de base : {TERRACLIMATE_VARIABLES}")
print(f"Variables avec agrégations temporelles : {TEMPORAL_VARS}")
print(f"Nombre de lags : {N_LAGS} mois")

Variables de base : ['pet', 'aet', 'ppt', 'tmax', 'tmin', 'soil', 'def', 'pdsi', 'vpd', 'ws']
Variables avec agrégations temporelles : ['ppt', 'soil', 'def', 'vpd']
Nombre de lags : 3 mois


---

## Étape 3 : Fonctions

In [4]:
def load_terraclimate_dataset():
    """
    Charge le dataset TerraClimate depuis Microsoft Planetary Computer.
    """
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )
    
    collection = catalog.get_collection("terraclimate")
    asset = collection.assets["zarr-abfs"]

    if "xarray:storage_options" in asset.extra_fields:
        ds = xr.open_zarr(
            asset.href,
            storage_options=asset.extra_fields["xarray:storage_options"],
            consolidated=True,
        )
    else:
        ds = xr.open_dataset(
            asset.href,
            **asset.extra_fields["xarray:open_kwargs"],
        )

    print(f"Dataset chargé ! Variables : {list(ds.data_vars)}")
    return ds

In [5]:
def filter_terraclimate(ds, var, start_date="2010-01-01", end_date="2015-12-31"):
    """
    Filtre le dataset TerraClimate pour une variable et une zone.
    Note: On commence en 2010 pour avoir les lags pour les données 2011.
    """
    ds_filtered = ds[var].sel(
        time=slice(start_date, end_date),
        lat=slice(-21.72, -35.18),
        lon=slice(14.97, 32.79)
    )
    
    df = ds_filtered.to_dataframe().reset_index()
    df['time'] = pd.to_datetime(df['time'])
    df = df.rename(columns={"lat": "Latitude", "lon": "Longitude", "time": "year_month"})
    
    # Convertir en période mensuelle
    df['year_month'] = df['year_month'].dt.to_period('M')
    
    return df

In [6]:
def build_kdtree(climate_df):
    """
    Construit un KD-Tree pour trouver les points climatiques les plus proches.
    """
    unique_coords = climate_df[['Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
    coords_radians = np.radians(unique_coords.values)
    tree = cKDTree(coords_radians)
    return tree, unique_coords

In [7]:
def extract_with_temporal_features(sites_df, climate_df, var_name, tree, unique_coords, n_lags=3):
    """
    Extrait une variable climatique avec ses agrégations temporelles.
    
    Pour chaque site et date, extrait :
    - Valeur du mois actuel
    - Valeurs des n mois précédents (lag1, lag2, lag3)
    - Cumul sur n+1 mois
    - Anomalie (écart à la moyenne du même mois sur toutes les années)
    """
    sites_df = sites_df.copy().reset_index(drop=True)
    
    # Trouver les coordonnées climatiques les plus proches
    site_coords = np.radians(sites_df[['Latitude', 'Longitude']].values)
    _, indices = tree.query(site_coords, k=1)
    sites_df['nearest_lat'] = unique_coords.iloc[indices]['Latitude'].values
    sites_df['nearest_lon'] = unique_coords.iloc[indices]['Longitude'].values
    
    # Convertir les dates
    sites_df['Sample Date'] = pd.to_datetime(sites_df['Sample Date'], dayfirst=True, errors='coerce')
    sites_df['year_month'] = sites_df['Sample Date'].dt.to_period('M')
    
    # Préparer le DataFrame climatique pour la jointure
    climate_pivot = climate_df.pivot_table(
        index=['Latitude', 'Longitude'],
        columns='year_month',
        values=var_name,
        aggfunc='first'
    )
    
    # Calculer la moyenne saisonnière (par mois de l'année)
    climate_df['month'] = climate_df['year_month'].apply(lambda x: x.month)
    seasonal_mean = climate_df.groupby(['Latitude', 'Longitude', 'month'])[var_name].mean().reset_index()
    seasonal_mean = seasonal_mean.rename(columns={var_name: f'{var_name}_seasonal_mean'})
    
    results = []
    
    for idx, row in sites_df.iterrows():
        lat, lon = row['nearest_lat'], row['nearest_lon']
        ym = row['year_month']
        month = ym.month
        
        result = {}
        
        try:
            # Valeur actuelle
            current_val = climate_pivot.loc[(lat, lon), ym] if ym in climate_pivot.columns else np.nan
            result[var_name] = current_val
            
            # Lags (mois précédents)
            lag_values = [current_val] if not pd.isna(current_val) else []
            for lag in range(1, n_lags + 1):
                lag_ym = ym - lag
                lag_val = climate_pivot.loc[(lat, lon), lag_ym] if lag_ym in climate_pivot.columns else np.nan
                result[f'{var_name}_lag{lag}'] = lag_val
                if not pd.isna(lag_val):
                    lag_values.append(lag_val)
            
            # Cumul sur n_lags+1 mois
            result[f'{var_name}_sum{n_lags+1}'] = np.sum(lag_values) if lag_values else np.nan
            
            # Moyenne sur n_lags+1 mois
            result[f'{var_name}_mean{n_lags+1}'] = np.mean(lag_values) if lag_values else np.nan
            
            # Anomalie (écart à la moyenne saisonnière)
            seasonal = seasonal_mean[(seasonal_mean['Latitude'] == lat) & 
                                     (seasonal_mean['Longitude'] == lon) & 
                                     (seasonal_mean['month'] == month)]
            if len(seasonal) > 0 and not pd.isna(current_val):
                seasonal_val = seasonal[f'{var_name}_seasonal_mean'].values[0]
                result[f'{var_name}_anomaly'] = current_val - seasonal_val
            else:
                result[f'{var_name}_anomaly'] = np.nan
                
        except (KeyError, IndexError):
            result[var_name] = np.nan
            for lag in range(1, n_lags + 1):
                result[f'{var_name}_lag{lag}'] = np.nan
            result[f'{var_name}_sum{n_lags+1}'] = np.nan
            result[f'{var_name}_mean{n_lags+1}'] = np.nan
            result[f'{var_name}_anomaly'] = np.nan
        
        results.append(result)
    
    return pd.DataFrame(results)

In [8]:
def extract_simple(sites_df, climate_df, var_name, tree, unique_coords):
    """
    Extrait une variable climatique simple (sans agrégations temporelles).
    """
    sites_df = sites_df.copy().reset_index(drop=True)
    
    # Trouver les coordonnées climatiques les plus proches
    site_coords = np.radians(sites_df[['Latitude', 'Longitude']].values)
    _, indices = tree.query(site_coords, k=1)
    sites_df['nearest_lat'] = unique_coords.iloc[indices]['Latitude'].values
    sites_df['nearest_lon'] = unique_coords.iloc[indices]['Longitude'].values
    
    # Convertir les dates
    sites_df['Sample Date'] = pd.to_datetime(sites_df['Sample Date'], dayfirst=True, errors='coerce')
    sites_df['year_month'] = sites_df['Sample Date'].dt.to_period('M')
    
    # Merge
    result = sites_df.merge(
        climate_df[['Latitude', 'Longitude', 'year_month', var_name]],
        left_on=['nearest_lat', 'nearest_lon', 'year_month'],
        right_on=['Latitude', 'Longitude', 'year_month'],
        how='left'
    )
    
    return pd.DataFrame({var_name: result[var_name].values})

---

## Étape 4 : Extraction pour les données d'entraînement

In [9]:
# Charger les données de qualité d'eau
Water_Quality_df = pd.read_csv("../data/raw/water_quality_training_dataset.csv")
print(f"Sites training : {len(Water_Quality_df)}")
display(Water_Quality_df.head())

Sites training : 9319


,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [10]:
# =============================================================================
# EXTRACTION TRAINING
# =============================================================================

print("1. Connexion à Microsoft Planetary Computer...")
ds = load_terraclimate_dataset()

# Initialiser le DataFrame final
Training_df = Water_Quality_df[['Latitude', 'Longitude', 'Sample Date']].copy()

# Cache pour réutilisation avec validation
tc_cache = {}

print("\n2. Extraction des variables...")

for var in tqdm(TERRACLIMATE_VARIABLES, desc="Variables"):
    # Filtrer les données (on prend depuis 2010 pour avoir les lags)
    climate_df = filter_terraclimate(ds, var, start_date="2010-01-01", end_date="2015-12-31")
    tc_cache[var] = climate_df
    
    # Construire le KD-Tree
    tree, unique_coords = build_kdtree(climate_df)
    
    if var in TEMPORAL_VARS:
        # Extraction avec agrégations temporelles
        var_df = extract_with_temporal_features(
            Water_Quality_df, climate_df, var, tree, unique_coords, n_lags=N_LAGS
        )
    else:
        # Extraction simple
        var_df = extract_simple(Water_Quality_df, climate_df, var, tree, unique_coords)
    
    # Ajouter au DataFrame final
    for col in var_df.columns:
        Training_df[col] = var_df[col].values

print(f"\nExtraction terminée : {len(Training_df)} lignes, {len(Training_df.columns)} colonnes")

1. Connexion à Microsoft Planetary Computer...
Dataset chargé ! Variables : ['aet', 'def', 'pdsi', 'pet', 'ppt', 'q', 'soil', 'srad', 'swe', 'tmax', 'tmin', 'vap', 'vpd', 'ws']

2. Extraction des variables...


Variables: 100%|██████████| 10/10 [21:31<00:00, 129.19s/it]


Extraction terminée : 9319 lignes, 37 colonnes


In [11]:
# Sauvegarder
output_path = os.path.join(OUTPUT_DIR, 'terraclimate_features_training_v2.csv')
Training_df.to_csv(output_path, index=False)
print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\terraclimate_features_training_v2.csv


In [12]:
# Aperçu
print(f"Colonnes créées ({len(Training_df.columns)}) :")
print(list(Training_df.columns))

print(f"\nAperçu des nouvelles features temporelles (ppt) :")
ppt_cols = [c for c in Training_df.columns if c.startswith('ppt')]
print(Training_df[ppt_cols].describe().round(2))

Colonnes créées (37) :
['Latitude', 'Longitude', 'Sample Date', 'pet', 'aet', 'ppt', 'ppt_lag1', 'ppt_lag2', 'ppt_lag3', 'ppt_sum4', 'ppt_mean4', 'ppt_anomaly', 'tmax', 'tmin', 'soil', 'soil_lag1', 'soil_lag2', 'soil_lag3', 'soil_sum4', 'soil_mean4', 'soil_anomaly', 'def', 'def_lag1', 'def_lag2', 'def_lag3', 'def_sum4', 'def_mean4', 'def_anomaly', 'pdsi', 'vpd', 'vpd_lag1', 'vpd_lag2', 'vpd_lag3', 'vpd_sum4', 'vpd_mean4', 'vpd_anomaly', 'ws']

Aperçu des nouvelles features temporelles (ppt) :
           ppt  ppt_lag1  ppt_lag2  ppt_lag3  ppt_sum4  ppt_mean4  ppt_anomaly
count  9319.00   9319.00   9319.00   9319.00   9319.00    9319.00      9319.00
mean     41.89     43.81     45.40     47.38    178.49      44.62        -1.86
std      44.91     47.23     49.10     49.59    137.17      34.29        30.61
min       0.00      0.00      0.00      0.00      0.00       0.00      -148.07
25%       7.30      7.00      7.10      8.50     70.90      17.73       -17.89
50%      28.40     30.10    

---

## Étape 5 : Extraction pour les données de validation

In [13]:
# Charger le template de soumission
Validation_df = pd.read_csv('../data/raw/submission_template.csv')
print(f"Sites validation : {len(Validation_df)}")

Sites validation : 200


In [14]:
# =============================================================================
# EXTRACTION VALIDATION
# =============================================================================

# Initialiser le DataFrame final
Validation_result_df = Validation_df[['Latitude', 'Longitude', 'Sample Date']].copy()

print("Extraction des variables...")

for var in tqdm(TERRACLIMATE_VARIABLES, desc="Variables"):
    # Réutiliser les données du cache
    climate_df = tc_cache[var]
    
    # Construire le KD-Tree
    tree, unique_coords = build_kdtree(climate_df)
    
    if var in TEMPORAL_VARS:
        var_df = extract_with_temporal_features(
            Validation_df, climate_df, var, tree, unique_coords, n_lags=N_LAGS
        )
    else:
        var_df = extract_simple(Validation_df, climate_df, var, tree, unique_coords)
    
    for col in var_df.columns:
        Validation_result_df[col] = var_df[col].values

print(f"\nExtraction terminée : {len(Validation_result_df)} lignes")

Extraction des variables...


Variables: 100%|██████████| 10/10 [08:01<00:00, 48.15s/it]


Extraction terminée : 200 lignes


In [15]:
# Sauvegarder
output_path = os.path.join(OUTPUT_DIR, 'terraclimate_features_validation_v2.csv')
Validation_result_df.to_csv(output_path, index=False)
print(f"Fichier créé : {output_path}")

Fichier créé : ../data/processed\terraclimate_features_validation_v2.csv


---

## Résumé des features créées

### Variables de base (10)
| Variable | Description |
|----------|-------------|
| pet | Évapotranspiration potentielle |
| aet | Évapotranspiration réelle |
| ppt | Précipitations |
| tmax | Température max |
| tmin | Température min |
| soil | Humidité du sol |
| def | Déficit hydrique |
| pdsi | Indice de sécheresse |
| vpd | Déficit de pression de vapeur |
| ws | Vitesse du vent |

### Agrégations temporelles (pour ppt, soil, def, vpd)
| Suffixe | Description | Exemple |
|---------|-------------|--------|
| `_lag1` | Valeur mois-1 | ppt_lag1 |
| `_lag2` | Valeur mois-2 | ppt_lag2 |
| `_lag3` | Valeur mois-3 | ppt_lag3 |
| `_sum4` | Cumul 4 derniers mois | ppt_sum4 |
| `_mean4` | Moyenne 4 derniers mois | ppt_mean4 |
| `_anomaly` | Écart à la moyenne saisonnière | ppt_anomaly |

### Total de features
- Variables simples : 6 (pet, aet, tmax, tmin, pdsi, ws)
- Variables avec temporel : 4 × 7 = 28 (ppt, soil, def, vpd avec 6 features chacune + la valeur de base)
- **Total : 34 features climatiques**

### Fichiers créés
| Fichier | Description |
|---------|-------------|
| terraclimate_features_training_v2.csv | Training avec agrégations temporelles |
| terraclimate_features_validation_v2.csv | Validation avec agrégations temporelles |